In [1]:
import multiprocessing as mp 
import random
import numpy as np
import osmnx as ox
import pandas as pd

np.random.seed(63)

bounding_box = (-118.45905139825884,34.05810307269331, -118.43480879942018,34.07964233309008 )
G = ox.graph.graph_from_bbox(bounding_box, network_type='drive')
#Gp = ox.projection.project_graph(G)

In [5]:
from traffix import *
import networkx as nx

In [11]:
G.nodes[21300209]

{'y': 34.059134,
 'x': -118.4426546,
 'highway': 'traffic_signals',
 'street_count': 4}

In [6]:
nodes, edges = ox.graph_to_gdfs(G)

In [7]:
nodes

,y,x,highway,street_count,geometry
osmid,,,,,
21300209,34.059134,-118.442655,traffic_signals,4,POINT (-118.44265 34.05913)
122568564,34.076642,-118.435383,NaN,3,POINT (-118.43538 34.07664)
122568567,34.077411,-118.436618,stop,3,POINT (-118.43662 34.07741)
122568569,34.077782,-118.439071,NaN,3,POINT (-118.43907 34.07778)
122579087,34.072235,-118.437247,NaN,3,POINT (-118.43725 34.07224)
...,...,...,...,...,...
10722587666,34.073910,-118.438461,NaN,5,POINT (-118.43846 34.07391)
10722642737,34.073749,-118.445334,NaN,3,POINT (-118.44533 34.07375)
10722642738,34.073828,-118.445221,NaN,3,POINT (-118.44522 34.07383)


In [5]:
def maxspeed_processing(maxspeed):
    if isinstance(maxspeed, list):
        maxspeed = maxspeed[0]
    if isinstance(maxspeed, str):
        return int(maxspeed.split()[0])
    return 25

In [30]:
edges

osmid       highway lanes  \
u           v           key                                               
21300209    123138234   0    [398527922, 398527669]       primary     8   
            122978983   0    [398527923, 397481335]       primary    10   
            122681080   0                 607194239      tertiary     2   
122568564   122568567   0                  13279976   residential     2   
            1467204393  0                  13279978   residential     2   
...                                             ...           ...   ...   
12614267731 12614267732 1     [810151348, 30724919]  unclassified     2   
            122725376   0                 810151348  unclassified     2   
12614267732 10722642738 0                  13350445  unclassified     3   
            12614267731 0     [810151348, 30724919]  unclassified     2   
            491992453   0                  30724919  unclassified     2   

                                     maxspeed                          name  \
u           v           key                                                   
21300209    123138234   0              35 mph            Wilshire Boulevard   
            122978983   0              35 mph            Wilshire Boulevard   
            122681080   0                 NaN                Glendon Avenue   
122568564   122568567   0                 NaN            Charing Cross Road   
            1467204393  0                 NaN            Charing Cross Road   
...                                       ...                           ...   
12614267731 12614267732 1    [15 mph, 20 mph]  Charles E. Young Drive North   
            122725376   0              20 mph  Charles E. Young Drive North   
12614267732 10722642738 0              20 mph                Westwood Plaza   
            12614267731 0    [15 mph, 20 mph]  Charles E. Young Drive North   
            491992453   0              15 mph  Charles E. Young Drive North   

                             oneway reversed      length  \
u           v           key                                
21300209    123138234   0     False    False  280.839261   
            122978983   0     False    False  117.826157   
            122681080   0     False     True  116.611066   
122568564   122568567   0     False    False  143.094961   
            1467204393  0     False     True   25.805766   
...                             ...      ...         ...   
12614267731 12614267732 1     False     True   14.509461   
            122725376   0     False    False  388.344837   
12614267732 10722642738 0      True    False   48.880708   
            12614267731 0     False    False   14.509461   
            491992453   0     False     True  460.152816   

                                                                      geometry  \
u           v           key                                                      
21300209    123138234   0    LINESTRING (-118.44265 34.05913, -118.44251 34...   
            122978983   0    LINESTRING (-118.44265 34.05913, -118.44268 34...   
            122681080   0    LINESTRING (-118.44265 34.05913, -118.44278 34...   
122568564   122568567   0    LINESTRING (-118.43538 34.07664, -118.43563 34...   
            1467204393  0    LINESTRING (-118.43538 34.07664, -118.4353 34....   
...                                                                        ...   
12614267731 12614267732 1    LINESTRING (-118.445 34.0734, -118.44493 34.07...   
            122725376   0    LINESTRING (-118.445 34.0734, -118.44503 34.07...   
12614267732 10722642738 0    LINESTRING (-118.44488 34.07349, -118.44495 34...   
            12614267731 0    LINESTRING (-118.44488 34.07349, -118.44493 34...   
            491992453   0    LINESTRING (-118.44488 34.07349, -118.4448 34....   

                            access tunnel  
u           v           key                
21300209    123138234   0      NaN    NaN  
            122978983   0      NaN    NaN  
            122681

In [7]:
intersections = {}
for node_id in nodes.index:
    intersections[node_id] = Intersection()

roads = []
for edge_id, edge_data in edges.iterrows():
    start_node_id, end_node_id, _ = edge_id
    start_intersection = intersections[start_node_id]
    end_intersection = intersections[end_node_id]
    
    length = edge_data['length']
    speed_limit = maxspeed_processing(edge_data['maxspeed'])
    lanes = edge_data['lanes'] 
    name = edge_data['name']
    
    road = Road(
        dt=1, 
        start=start_intersection,
        end=end_intersection,
        num_cars=20,
        speed_limit=speed_limit,
        length=length,
        lanes=lanes,
        name=name
    )
    roads.append(road)

traffic_map = Map(intersections=list(intersections.values()), roads=roads)

In [8]:
traffic_map.roads[2].speed_limit

25

In [29]:
traffic_map.roads[0].num_cars

21.37686955607093

In [31]:
intersections

{21300209: <__main__.Intersection at 0x2104507a850>,
 122568564: <__main__.Intersection at 0x2103f506e10>,
 122568567: <__main__.Intersection at 0x21046620d50>,
 122568569: <__main__.Intersection at 0x21047a352d0>,
 122579087: <__main__.Intersection at 0x21047a34790>,
 122579090: <__main__.Intersection at 0x21047a35bd0>,
 122579102: <__main__.Intersection at 0x21047a349d0>,
 122584728: <__main__.Intersection at 0x21047a36050>,
 122584732: <__main__.Intersection at 0x21047a36110>,
 122584740: <__main__.Intersection at 0x21047a361d0>,
 122584755: <__main__.Intersection at 0x21047a35f90>,
 122584764: <__main__.Intersection at 0x21047a34910>,
 122584789: <__main__.Intersection at 0x21047a34c10>,
 122613440: <__main__.Intersection at 0x21047a34cd0>,
 122613443: <__main__.Intersection at 0x21047a34e50>,
 122624759: <__main__.Intersection at 0x21047a34f10>,
 122643255: <__main__.Intersection at 0x21047a35090>,
 122643258: <__main__.Intersection at 0x21047a34d90>,
 122643262: <__main__.Interse

In [27]:
traffic_map.update()

Wilshire Boulevard #####################
Wilshire Boulevard ###################
Glendon Avenue ###################
Charing Cross Road ########################
Charing Cross Road ###############
Charing Cross Road ####################
Charing Cross Road ###################
Loring Avenue ###################
Charing Cross Road ####################
Hilgard Avenue ###############
Hilgard Avenue ######################
Warner Avenue #################
Hilgard Avenue ######################
Hilgard Avenue #################
Warner Avenue ###################
Warner Avenue ###################
Dalehurst Avenue ####################
Strathmore Drive ####################
Warner Avenue ###################
Loring Avenue ###################
Veteran Avenue #####################
West Sunset Boulevard ###################
West Sunset Boulevard ##################
Cashmere Street ###################
Veteran Avenue ####################
Veteran Avenue ###################
Gayley Avenue ##################
Veteran A